## BD5append_trackinfo_point

BD5append_trackinfo_point is a sample code to demonstrate how to append trackInfo information into a BD5 file. The code opens a BD5 file of a time series and calculate the tracking information between objects of different time points. After finding the closest objects between two time points, we re-open the BD5 file and append the information into trackInfo dataset.

This sample code requires the user to install several packages. They are:

- h5py - https://www.h5py.org/
- Numpy - https://numpy.org/

In [1]:
# Please consult the official websites for installation for your particular Operating system and environment.
# The following commands installed the necessary packages for juypter notebook in a Ubuntu docker container as a sample for your reference.

# Uncomment the # tag if you want the commands below to install those packages.

#!pip install h5py

In [2]:
# Original Author: Koji Kyoda
# Contributor: Ken ho 
# Date: 7 July 2020
# sample code for appending trackInfo into BD5 file


import sys
import h5py
import numpy as np
import matplotlib.pyplot as plt
import math

### Opening the BD5 file containing timeseries information

In [3]:
inputfile = "sample_timeseries_notrack_bd5.h5"
fi = h5py.File(inputfile, "r")
groups = fi['data']

Under BD5 data group, there are time sequence and other groups like objectDef, scaleUnit, etc.

In [4]:
groups.keys()

<KeysViewHDF5 ['0', '1', 'objectDef', 'scaleUnit']>

 We create a list that we want to exclude

In [5]:
exclude_list = ['objectDef', 'scaleUnit', 'trackInfo']

### Finding a list of timeseries 
by excluding those members in the exclude_list

In [6]:
timeseries = set(groups.keys()) - set(exclude_list)
timeseries

{'0', '1'}

### Creating a numpy array "track_arr" 
The numpy array is used to store tracking information, 'from', 'to' and 'distance'

In [7]:
track_descr = np.dtype({'names': ['from','to', 'dis'], 'formats': ['S32', 'S32', 'f8']})
track_arr = np.empty([0, 1], dtype=track_descr)

### Creating a function "min_dis"
The function is to find a list of minimal distance between objects in different time point given a track_arr

In [8]:
def min_dis(trackarray):
    min_dis = 1000.0
    min_list = []
    for i in trackarray:
#        print(i['dis'])
        if i['dis'] < min_dis:
            min_dis = i['dis']
            min_list = (i['from'], i['to'], i['dis'])
    return min_list

### The following routine tries to track every objects distance between the current time point over the previsous time point.
* It takes each object in each time point and calculate the distance between the objects in the current time point and the objects in the previous time points. It put it in a numpy array track_arr.
* It then uses the function min_dis to return a list of objects with minumum distance between two time points and return the min_track_list

In [9]:
min_track_lst = []
for ti in timeseries:
    name = "/data/" + ti + "/object/0"
    dset = fi[name]
    for i in range(dset.len()):
        # track_arr to collect all the pairs between the current time series objects and the previous ones.
        track_arr = np.empty([0, 1], dtype=track_descr) 
        prev_subname = int(ti)-1
        if prev_subname >= 0: # previous time index is larger than 0
            prev_name = "/data/" + str(prev_subname) + "/object/0"
            prev_dset = fi[prev_name]       
            curr_ID = dset[i, 'ID']
            prev_ID = ""
            for j in range(prev_dset.len()):
                prev_ID = prev_dset[j, 'ID']
                dis = math.sqrt((dset[i, 'x'] - prev_dset[j, 'x']) * (dset[i, 'x'] - prev_dset[j, 'x']) + (dset[i, 'y'] - prev_dset[j, 'y']) * (dset[i, 'y'] - prev_dset[j, 'y']))
                track_arr = np.append(track_arr, np.array([(prev_ID, curr_ID, dis)], dtype=track_descr))
            min_track_lst.append(min_dis(track_arr))
print(min_track_lst)

[(b'o000000', b'o001000', 23.67861135586687), (b'o000001', b'o001001', 26.749393064084185)]


### Closing the BD5 file

In [10]:
fi.close()

## Appending trackInfo information to existing BD5 file
Instead of adding the original sample file, we create a copy to demonstrate how to append trackInfo.

In [11]:
# using system command to create a new copy to demonstrate how to append to a BD5 file
!cp sample_timeseries_notrack_bd5.h5 sample_timeseries_trackInfo_bd5.h5

In [12]:
# Alternatively a duplicate of the file can be achieved by uncomment the python codes below.

#inputfile = "sample_timeseries_notrack_bd5.h5"
#fi = h5py.File(inputfile, "r")
#groups = fi['data']
#outputfile ="sample_timeseries_trackInfo_bd5.h5"
#fo = h5py.File(outputfile, 'w')
#fi.copy('data', fo)
#fi.close

### Procedures to append trackInfo
by using the min_track_lst that we created earler.

In [13]:
# append a new trackInfo dataset to the BD5 file
outputfile ="sample_timeseries_trackInfo_bd5.h5"
fo = h5py.File(outputfile, 'a')

# creating trackInfo numpy array
trackInfo_descr = np.dtype({'names': ['from','to'], 'formats': ['S32', 'S32']})
arr = np.empty([0, 0], dtype=trackInfo_descr)

for i in min_track_lst:
    arr = np.append(arr, np.array([(i[0], i[1])], dtype=trackInfo_descr))

# appending the trackInfo numpy array into the BD5 file
fo.create_dataset('/data/trackInfo', data=arr)

# closing the BD5 file
fo.close()
